In [1]:
# Import libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

C:\Users\MrAzuka\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\MrAzuka\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# Get dataset
train_dataset = pd.read_csv("C:\\Users\MrAzuka\Desktop\Hackacthon_Practice\Train_v2.csv")
test_dataset = pd.read_csv("C:\\Users\MrAzuka\Desktop\Hackacthon_Practice\Test_v2.csv")

In [3]:
train_dataset["unique_ids"] = train_dataset['uniqueid'] +'x' + train_dataset['country']
test_dataset_2 = test_dataset.copy()

test_dataset["unique_ids"] = test_dataset['uniqueid'] +'x' + test_dataset['country']

train_dataset.drop(['uniqueid', 'country'], axis=1, inplace=True)
test_dataset.drop([ 'uniqueid','country'], axis=1, inplace=True)

train_dataset = train_dataset.set_index('unique_ids')
test_dataset = test_dataset.set_index('unique_ids')


In [4]:
# Create dummies
# Using LabelEncoder
# Convert target label to numerical Data
label = LabelEncoder()
train_dataset['bank_account'] = label.fit_transform(train_dataset['bank_account'])


In [5]:
# Data preprocessing 
# convert categorical features to numerical features
# categorical features to be converted by One Hot Encoding

categ = ['relationship_with_head', 'marital_status', 'education_level', 'job_type']
# One Hot Encoding conversion
train_dataset = pd.get_dummies(train_dataset, prefix_sep='_', columns = categ)

test_dataset = pd.get_dummies(test_dataset, prefix_sep='_', columns = categ)

In [6]:
# Labelncoder conversion
train_dataset['location_type'] = label.fit_transform(train_dataset['location_type'])
train_dataset['cellphone_access'] = label.fit_transform(train_dataset['cellphone_access'])
train_dataset['gender_of_respondent'] = label.fit_transform(train_dataset['gender_of_respondent'])

In [7]:
test_dataset['location_type'] = label.fit_transform(test_dataset['location_type'])
test_dataset['cellphone_access'] = label.fit_transform(test_dataset['cellphone_access'])
test_dataset['gender_of_respondent'] = label.fit_transform(test_dataset['gender_of_respondent'])

X = train_dataset.drop(['year', 'bank_account'], axis=1)
y = train_dataset['bank_account']

X_test = test_dataset.drop(['year'], axis=1)

In [8]:
# Splitting the dataset into training and test
X_train, X_val, y_train, y_val = train_test_split(X,y,  test_size = 0.2, random_state=42)

In [9]:
# Using the RandomForestRegressor
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [10]:
fin_preds = forest_model.predict(X_val)

In [11]:
forest_model.score(X_train,y_train)

0.7802796886327625

In [12]:
# Using the XGBOOST
classifier = XGBClassifier()
classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [13]:
y_pred = classifier.predict(X_val)

In [14]:
# making the confusion matrix
con_mat = confusion_matrix(y_val, y_pred)
con_mat

array([[3932,  131],
       [ 418,  224]], dtype=int64)

In [15]:
(3932+224)/(3932+131+418+224)

0.8833156216790649

In [16]:
# To check the accuracy of my model
# using cross validation method
accuracy = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)

In [17]:
accuracy.mean()

0.8811308735885055

In [18]:
accuracy.std()

0.0068608879885167255

In [19]:
# Get the predicted result for the test Data
test_dataset.bank_account = classifier.predict(X_test)

C:\Users\MrAzuka\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [20]:
test_dataset_2.bank_account = classifier.predict(X_test)

C:\Users\MrAzuka\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [21]:
# create submission DataFrame
submission_df3 = pd.DataFrame({"uniqueid": test_dataset_2["uniqueid"] + " x " + test_dataset_2["country"],

"bank_account": test_dataset_2.bank_account})

In [22]:
submission_df3.head()

,uniqueid,bank_account
0,uniqueid_6056 x Kenya,1
1,uniqueid_6060 x Kenya,1
2,uniqueid_6065 x Kenya,0
3,uniqueid_6072 x Kenya,0
4,uniqueid_6073 x Kenya,0


In [23]:
# Create submission csv file
submission_df3.to_csv('SubFile.csv', index = False)